In [1]:
import numpy as np
import pandas as pd
import math
import sys, os
#import seaborn as sns
#from scipy import stats
#from pathlib import Path
#import matplotlib.pyplot as plt
#from sklearn.model_selection import KFold
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
#from sklearn.metrics import mean_squared_error
#from sklearn.metrics import mean_squared_error
#from sklearn.preprocessing import MinMaxScaler
#from sklearn.datasets import fetch_california_housing
#import math
from tqdm import tqdm
from sklearn.feature_extraction.text import CountVectorizer
import optuna
from optuna.samplers import TPESampler
#import lightgbm as lgbm
#from xgboost import XGBRegressor
import xgboost as xgb
#from catboost import CatBoostRegressor
#from lightgbm.sklearn import LGBMRegressor
#from sklearn.ensemble import RandomForestRegressor
import pickle
from sklearn.metrics import roc_curve, auc

In [2]:
sys.path.append(os.path.realpath('..'))

In [3]:
train_df = pd.read_csv(r"D:\source\repos\Kaggle_Tabular_Playground_Series-ML\Jan-2023-S2\data\train.csv")
test_df = pd.read_csv(r"D:\source\repos\Kaggle_Tabular_Playground_Series-ML\Jan-2023-S2\data\test.csv")
sample_sub = pd.read_csv(r"D:\source\repos\Kaggle_Tabular_Playground_Series-ML\Jan-2023-S2\data\sample_submission.csv")

In [4]:
columns_to_vectorize = ['gender', 'ever_married', 'work_type', 'Residence_type', 'smoking_status']
for vector_target in columns_to_vectorize:
    vectorizer = CountVectorizer()
    vectorizer.fit_transform(train_df[vector_target])
    train_df[f'{vector_target}_v'] = vectorizer.transform(train_df[vector_target]).toarray().argmax(axis=1)[:,None]
    vectorizer.fit_transform(test_df[vector_target])
    test_df[f'{vector_target}_v'] = vectorizer.transform(test_df[vector_target]).toarray().argmax(axis=1)[:,None]

In [5]:
features = ['id', 'age', 'hypertension', 'heart_disease', 'avg_glucose_level', 'bmi', 'gender_v', 'ever_married_v', 'work_type_v', 'Residence_type_v', 'smoking_status_v']
target = ['stroke']

In [6]:
scaler = MinMaxScaler().fit(train_df[features])
X = scaler.transform(train_df[features])
X_test = scaler.transform(test_df[features])

scaler = MinMaxScaler().fit(train_df[target])
Y = scaler.transform(train_df[target])

In [7]:
y_min = Y.min()
y_max = Y.max()

print(y_min, y_max)

def my_rmse(y_true, y_hat):
    y_true[y_true < y_min] = y_min
    y_true[y_true > y_max] = y_max
    
    y_hat[y_hat < y_min] = y_min
    y_hat[y_hat > y_max] = y_max
    
    y_true_nan = np.isnan(y_true)
    y_hat_nan = np.isnan(y_hat)
    
    if y_true_nan.sum() > 0:
        print(y_true_nan.sum())
        np.where(y_true_nan, np.ma.array(y_true, mask=np.isnan(y_true)).mean(axis=0), y_true)
    if y_hat_nan.sum() > 0:
        print(y_hat_nan.sum())
        np.where(y_hat_nan, np.ma.array(y_hat, mask=np.isnan(y_hat)).mean(axis=0), y_hat)
    
    return mean_squared_error(y_true, y_hat, squared=False)

0.0 1.0


In [8]:
def xgb_objective(trial):
    # Split the train data for each trial.
    X_train, X_valid, y_train, y_valid = train_test_split(X, Y, stratify=Y, test_size=0.4)

    param_grid = {
        'max_depth': trial.suggest_int('max_depth', 4, 20),
        'n_estimators': trial.suggest_int('n_estimators', 2, 100, 1), 
        'eta': trial.suggest_float('eta', 0.0007, 0.113), 
        'subsample': trial.suggest_float('subsample', 0.1, 1),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.1, 1),
        'reg_lambda': trial.suggest_float('reg_lambda', 1, 40), 
    } 
    
    reg = xgb.XGBModel(
        # These parameters should help with trial speed.
        objective='binary:logistic',
        tree_method='gpu_hist',
        booster='gbtree',
        predictor='gpu_predictor',
        n_jobs=4,
        eval_metric='auc',
        **param_grid
    )
    
    reg.fit(X_train, y_train, eval_set=[(X_valid, y_valid)], verbose=False)
    
    preds = reg.predict(X_valid)
    fpr, tpr, _ = roc_curve(y_valid, preds)
    roc_auc = auc(fpr, tpr)

    xgb_ranks[roc_auc] = reg

    print(roc_auc)
    return roc_auc

In [9]:
xgb_ranks = {}
    
train_time = 1 * 60 * 60
study = optuna.create_study(direction='maximize', sampler=TPESampler(), study_name='XGBRegressor')
study.optimize(xgb_objective, timeout=train_time)

[I 2023-01-16 08:57:03,517] A new study created in memory with name: XGBRegressor
[I 2023-01-16 08:57:04,153] Trial 0 finished with value: 0.8888273416227961 and parameters: {'max_depth': 12, 'n_estimators': 64, 'eta': 0.1070254593837115, 'subsample': 0.9865682232151324, 'colsample_bytree': 0.45126970181863757, 'reg_lambda': 37.45707980982996}. Best is trial 0 with value: 0.8888273416227961.


0.8888273416227961


[I 2023-01-16 08:57:04,394] Trial 1 finished with value: 0.8563568747697589 and parameters: {'max_depth': 11, 'n_estimators': 30, 'eta': 0.06751339034067358, 'subsample': 0.5387623190506561, 'colsample_bytree': 0.8607969073703937, 'reg_lambda': 12.866592531617766}. Best is trial 0 with value: 0.8888273416227961.


0.8563568747697589


[I 2023-01-16 08:57:04,829] Trial 2 finished with value: 0.8959768516429528 and parameters: {'max_depth': 19, 'n_estimators': 72, 'eta': 0.09173244600738494, 'subsample': 0.6298705286724017, 'colsample_bytree': 0.4418802598009858, 'reg_lambda': 22.259182906217656}. Best is trial 2 with value: 0.8959768516429528.
[I 2023-01-16 08:57:04,957] Trial 3 finished with value: 0.812277882651326 and parameters: {'max_depth': 5, 'n_estimators': 16, 'eta': 0.07999468544869776, 'subsample': 0.5981946449499476, 'colsample_bytree': 0.29074775284450677, 'reg_lambda': 31.380863174302327}. Best is trial 2 with value: 0.8959768516429528.


...

[I 2023-01-16 09:04:34,928] Trial 511 finished with value: 0.8810444372757781 and parameters: {'max_depth': 16, 'n_estimators': 84, 'eta': 0.030901617050496932, 'subsample': 0.4998611556121546, 'colsample_bytree': 0.7833741586908287, 'reg_lambda': 23.133819214498654}. Best is trial 311 with value: 0.9077621616088283.


0.8810444372757781


[I 2023-01-16 09:04:35,892] Trial 512 finished with value: 0.8615297634721728 and parameters: {'max_depth': 17, 'n_estimators': 55, 'eta': 0.1092015288111785, 'subsample': 0.6274593995942306, 'colsample_bytree': 0.5586354664223647, 'reg_lambda': 32.634718747477976}. Best is trial 311 with value: 0.9077621616088283.


0.8615297634721728


[W 2023-01-16 09:04:36,951] Trial 513 failed because of the following error: KeyboardInterrupt()
Traceback (most recent call last):
  File "D:\source\repos\venv\Python310\lib\site-packages\optuna\study\_optimize.py", line 196, in _run_trial
    value_or_values = func(trial)
  File "C:\Users\ryans\AppData\Local\Temp\ipykernel_42812\160111865.py", line 25, in xgb_objective
    reg.fit(X_train, y_train, eval_set=[(X_valid, y_valid)], verbose=False)
  File "D:\source\repos\venv\Python310\lib\site-packages\xgboost\core.py", line 620, in inner_f
    return func(**kwargs)
  File "D:\source\repos\venv\Python310\lib\site-packages\xgboost\sklearn.py", line 1025, in fit
    self._Booster = train(
  File "D:\source\repos\venv\Python310\lib\site-packages\xgboost\core.py", line 620, in inner_f
    return func(**kwargs)
  File "D:\source\repos\venv\Python310\lib\site-packages\xgboost\training.py", line 186, in train
    if cb_container.after_iteration(bst, i, dtrain, evals):
  File "D:\source\repos\v

In [10]:
xgb_ranks_bytes = pickle.dumps(xgb_ranks)
with open("rendered_data/xgb_clsfs_bytes.hex", "wb") as binary_file:
    binary_file.write(xgb_ranks_bytes)

In [11]:
percent = math.ceil(len(xgb_ranks.keys()) * .1)
top = sorted(list(xgb_ranks.keys()))[-percent:]

train_preds = []
test_preds = []
for key in tqdm(top):
    train_preds.append(xgb_ranks[key].predict(X))
    test_preds.append(xgb_ranks[key].predict(X_test))
    
train_final_preds = np.stack(train_preds).mean(0)
test_final_preds = np.stack(test_preds).mean(0)

100%|██████████████████████████████████████████████████████████████████████████████████| 52/52 [00:01<00:00, 45.27it/s]


In [12]:
XGB_train_submission = pd.DataFrame(data={'id': train_df.id, 'stroke': train_final_preds})
XGB_train_submission.to_csv(fr'rendered_data/XGB_train_submission.csv', index=False)

XGB_test_submission = pd.DataFrame(data={'id': test_df.id, 'stroke': test_final_preds})
XGB_test_submission.to_csv(fr'rendered_data/XGB_test_submission.csv', index=False)